In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import subprocess
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from Bio.Seq import Seq
from dask.distributed import wait
from dask.distributed import Client
import os

sys.path.append("../scripts")
%aimport initial_map
%aimport map_refiner
%aimport complexity
%aimport finder
%aimport preprocess

import glob
from tqdm import tqdm  # progress bar

In [4]:
import duckdb
from concurrent.futures import ThreadPoolExecutor, as_completed

# intial map

In [6]:
glob.glob("/global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_step1*.assembled.*")

['/global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_step1_S1.fastq.gz.assembled.fastq']

In [7]:
mapper = initial_map.BarcodeMapper(seq_file = "/global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_step1_S1.fastq.gz.assembled.fastq",
                        bc_names = ["AD", "AD_BC", "RPTR_BC"],
                        preceders = ["AGGAGCA", "GGCCTC", "CTCGAG"],
                        posts = ["TGATAAG", "GGGCCC", "GGCCGC"],
                        lengths = [120, 11, 14],
                      reverse_complement = False,
                      txt_path = "../data/plasm_step1_reads_shortened.txt")
mapped_df = mapper.create_map()
mapped_df.head()

fastq file detected. Will shorten to txt file and save at ../data/plasm_step1_reads_shortened.txt


,AD,AD_qual,AD_BC,AD_BC_qual,RPTR_BC,RPTR_BC_qual,Designed
0,AATCATCGGGATTTTAACAAGAACATACTGAATATTACTTGTTATA...,True,GCCGCGCCAGC,True,ACCACCTTCATTGA,True,0
1,<NA>,False,ACGAGAGATGG,True,CTAAGATAATCACG,True,0
2,GAACGGCATAAGGAGAACGTACAGAATAAGATAAAGCAAATCGATA...,True,CACGACACCAC,True,CAGTGGACCCTGTG,True,0
3,GTTATCTGCATAGATACCTTCCTACAGACCAAAAAGTACATGCTCA...,True,TGCCATCCCTT,True,GGATTTCTAGTCCC,True,0
4,AAGGAAAACTTCCTTACAGATATTAACGAGGCTTCCAAGAAGAACC...,True,CCTCTTTCTGC,True,GTCGTGGCCACCAC,True,0


In [17]:
with ProgressBar():
    mapped_df.to_parquet(
        '/global/scratch/projects/fc_mvslab/OpenProjects/Sanjana/TREBL/data/Plasm_step1_S1.parquet',
        engine='pyarrow',
        write_index=False
    )

[########################################] | 100% Completed | 117.98 s


# refining map

In [8]:
! rm /global/scratch/projects/fc_mvslab/OpenProjects/Sanjana/TREBL/duckdb/Plasm_step1_S1.db

In [5]:
refiner = map_refiner.MapRefiner(db_path = "/global/scratch/projects/fc_mvslab/OpenProjects/Sanjana/TREBL/duckdb/Plasm_step1_S1.db",
                     cols = ["AD", "AD_BC", "RPTR_BC"],
                     reads_threshold = 5,
                     column_pairs = [("AD", "AD_BC"), ("AD_BC", "RPTR_BC")],
                    design_check = False)

In [28]:
refiner.refine_map_from_parquet('/global/scratch/projects/fc_mvslab/OpenProjects/Sanjana/TREBL/data/Plasm_step1_S1.parquet/*')

Refining maps:  20%|██        | 1/5 [00:01<00:06,  1.70s/step]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Refining maps:  40%|████      | 2/5 [00:06<00:09,  3.26s/step]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Refining maps: 100%|██████████| 5/5 [00:12<00:00,  2.54s/step]


In [6]:
refiner.get_map_df('map5_thresholded')

,AD,AD_BC,RPTR_BC,count
0,AATAATGACGTCATAACGAAGACGTTGTCAAGGATGAGACTATCAA...,ATCGCGTCTAT,CGTTCGCTGTGCCT,206
1,AAAGAGGTGATGTTTGAAAAGTACAACAAGCTGCTTTCGTCTTATA...,GCGGGAGGTCA,AGCTCACGGCGAAA,185
2,GAGAAGAACAACCTGAACGATTTGCTCTCGTAGATAACGATAAATC...,TCAGAACAACG,GCACGGACCGAGAC,135
3,CCTAAGAATGAGATCGACAAAAACGACGAGGAAAATCTTAAAGAAT...,CCAGCAATACG,TGCTTGTGTGACTT,134
4,TCGATATTCCCGACCAGGACTTTCCCTATATAATTGCGTGGATTCT...,ACACATCGTCT,GAACACCTGCGGCT,131
...,...,...,...,...
175791,ATAGCAGGGTTGAAACGGATGGGCATCTCAACGTTGGGCGGCAAGG...,TGGGTGGGGCA,CAGATACTCAAGCG,6
175792,TTCTCCAAAAATGGCAGCATGAGTTCTATACCTAGTGACTACAACA...,AACTAGTGAGC,CAAACGACGCGAAT,6
175793,TCCGAGTATGAGGAGTTAGACATAAACGACATCTATGTTCCACGGG...,CAGCCGACACC,AAGGCACTGTCTCT,6
175794,TCCGAGGCTGCTGGGGGTGCGTCGGAAGGAGTGTTTAGAGAAGCAG...,ACCTGCTTGCT,CCCCTACAACGACT,6


# Step 2

In [14]:
plasm_step2_paths = ["/global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_RP_step2_S4.fastq.gz.assembled.fastq",
                    "/global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_AD_step2_S3.fastq.gz.assembled.fastq"]
plasm_step2_paths

['/global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_RP_step2_S4.fastq.gz.assembled.fastq',
 '/global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_AD_step2_S3.fastq.gz.assembled.fastq']

In [27]:
plasm_AD_step2_dd = preprocess.load_and_shorten_files(plasm_step2_paths,
                                  reverse_complement=True,
                                  txt_path = "../data/plasm_step2_AD.txt")

Shortening /global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_RP_step2_S4.fastq.gz.assembled.fastq -> ../data/plasm_step2_AD_0.txt
Shortening /global/scratch/projects/fc_mvslab/data/sequencing/czb_new_sept2025/Marissa/results/Plasm_AD_step2_S3.fastq.gz.assembled.fastq -> ../data/plasm_step2_AD_1.txt
('../data/plasm_step2_AD_0.txt', '../data/plasm_step2_AD_1.txt')


In [28]:
plasm_AD_step2_dd = finder.add_barcode(plasm_AD_step2_dd, "RPTR_BC", "CTCGAG", "GGCCGC", 14)
plasm_AD_step2_dd = finder.add_barcode(plasm_AD_step2_dd, "AD_BC", "GGCCTC", "GGGCCC", 11)
plasm_AD_step2_dd.head()

,sequence,RPTR_BC,RPTR_BC_qual,AD_BC,AD_BC_qual
0,ATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCCAC...,<NA>,False,<NA>,False
1,ATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCCAC...,<NA>,False,<NA>,False
2,CTATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCC...,CACATAACCCGTCA,True,<NA>,False
3,CTATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCC...,CCACTCTCGGATCT,True,<NA>,False
4,TATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCCA...,<NA>,False,<NA>,False


# next steps:

- add to compexity.py
- check one bc type at a type
- how many RP bc from map1? of total RP bc
- how many AD bc from map1? of total AD bc
- how many unique AD bc-RP bc pairs are in map 1? of total AD bc-RP bc pairs

In [29]:
# How many RP BC from map 1 of total RP BC

In [30]:
preprocess.save_parquet(plasm_AD_step2_dd, "../output/plasm_AD_step2_dd_parquet")

[########################################] | 100% Completed | 194.28 s


In [31]:
con = duckdb.connect("/global/scratch/projects/fc_mvslab/OpenProjects/Sanjana/TREBL/duckdb/Plasm_step1_S1.db")

In [35]:
con.execute("CREATE OR REPLACE TABLE step2_complexity AS SELECT * FROM read_parquet('../output/plasm_AD_step2_dd_parquet/*')")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [37]:
# How many RP BC from map 1 of total RP BC?
con.execute("""SELECT * FROM step2_complexity LIMIT 5""").df()

,sequence,RPTR_BC,RPTR_BC_qual,AD_BC,AD_BC_qual
0,ATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCCAC...,None,False,None,False
1,ATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCCAC...,None,False,None,False
2,CTATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCC...,CACATAACCCGTCA,True,None,False
3,CTATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCC...,CCACTCTCGGATCT,True,None,False
4,TATGGTGATGGTCCAGTCTTGTTACCAGACAACCATTACTTATCCA...,None,False,None,False


In [47]:
def count_multiple_bc(con, bc_columns):
    """
    Count unique barcodes for multiple columns and return a summary table.

    Parameters:
        con: DuckDB connection
        bc_columns: list of column names (str) to count

    Returns:
        pandas.DataFrame with one row per BC column
    """
    results = []

    for bc in bc_columns:
        query = f"""
        SELECT
            (SELECT COUNT(DISTINCT {bc}) FROM map5_thresholded) AS unique_bc_in_map5,
            (SELECT COUNT(DISTINCT {bc}) FROM step2_complexity) AS unique_bc_in_step2,
            (SELECT COUNT(DISTINCT s.{bc})
             FROM step2_complexity s
             JOIN map5_thresholded m
               ON s.{bc} = m.{bc}) AS unique_bc_in_both;
        """
        df = con.execute(query).df()
        df.insert(0, "BC_type", bc)  # Add column name as first column
        results.append(df)

    # Combine all rows into a single DataFrame
    return pd.concat(results, ignore_index=True)

In [49]:
count_multiple_bc(con, ["AD_BC", "RPTR_BC"])

,BC_type,unique_bc_in_map5,unique_bc_in_step2,unique_bc_in_both
0,AD_BC,175674,645537,86955
1,RPTR_BC,175796,771465,85882


In [ ]:
# Do the RPTR BCs also match an AD BC seen in the step?

In [ ]:
# ie for RPTR BC, check if RPTR BC in map
# Then, use map to get AD BC corresp to RP BC
# Then check if those AD BCs are in step 2
# Then count unqiue RP BCs remaining